In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
import gradio as gr
import torch
from transformers import pipeline


In [3]:
def chunk_text(input_text, chunk_size=1024):
    return [input_text[i:i + chunk_size] for i in range(0, len(input_text), chunk_size)]

In [4]:
model_path = r"model\models--sshleifer--distilbart-cnn-12-6\snapshots\a4f8f3ea906ed274767e9906dbaede7531d660ff"
text_summary = pipeline("summarization", model=model_path, torch_dtype=torch.float16)

def summary(input_text):
    chunks = chunk_text(input_text)
    summaries = [text_summary(chunk)[0]['summary_text'] for chunk in chunks]
    return " ".join(summaries)

Device set to use cuda:0


In [5]:

def get_video_id(url):
    """
    Extract the video ID from the YouTube URL.
    """
    if 'youtube.com/watch?v=' in url:
        return url.split('v=')[1].split('&')[0]
    elif 'youtu.be/' in url:
        return url.split('youtu.be/')[1]
    else:
        raise ValueError("Invalid YouTube URL")


In [6]:
def get_transcript(video_url):
    """
    Fetches the transcript for a YouTube video and summarizes it.
    """
    try:
        video_id = get_video_id(video_url)
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine transcript entries into a single string
        transcript_text = " ".join(entry['text'] for entry in transcript)
        summary_text = summary(transcript_text)
        return summary_text
    except TranscriptsDisabled:
        return "Transcripts are disabled for this video."
    except NoTranscriptFound:
        return "No transcript found for this video."
    except Exception as e:
        return f"An error occurred: {e}"

In [7]:
def main():
    """
    Main function to run the script.
    """
    gr.close_all()

    demo = gr.Interface(fn=get_transcript, inputs=[gr.Textbox(label="Input the YouTube URL summarize", lines=1)],
                     outputs=[gr.Textbox(label="Summarized the youtube URL", lines=4)],
                     title="@GenAI project 2 : YouTube URL Summarizer",
                     description="This Applicatio will be used to summarize the YouTube URL")
    demo.launch(share=True)


In [8]:
if __name__ == "__main__":
    main()

Closing server running on port: 7861
* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
